In [40]:
import numpy as np
from evaluation import EvaluationMetrics
from dataloader import ImageDataset
from panet import PanNet
import torch
from torch.utils.data import DataLoader

In [38]:
model = PanNet()
model.load_state_dict(torch.load('/Users/aniruddha/Downloads/panet_model_4.dms'))

In [32]:
dataset = ImageDataset(root_dir= '/Users/aniruddha/Downloads/Test', dataset_type= 3)

batch_size = 3
dataloader = DataLoader(dataset=dataset, batch_size= batch_size)

In [39]:
bbox_global = None
bbox_pred_global = None

start = True
count = 0
for batch in dataloader:
    img = batch['image']
    bbox = batch['labels']
    bbox_pred = model(img)
    count += 1
    if start:
        bbox_global = bbox
        bbox_pred_global = bbox_pred
        start = False
    else:
        bbox_global = torch.cat((bbox_global, bbox), dim = 0)
        bbox_pred_global = torch.cat((bbox_pred_global, bbox_pred), dim = 0)
    print(count)

1
2
3
4


KeyboardInterrupt: 

In [41]:
eval1 = EvaluationMetrics()
map_score = eval1.mean_average_precision(bbox = bbox_global, bbox_pred = bbox_pred_global)